In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from sklearn.decomposition import PCA
import keras

Using TensorFlow backend.


In [10]:
import pickle

In [2]:
# Use header to load my csv file with variable names
header = ['label','rndm','weight']
# boosted frame
header += ['W_B_pt','W_B_eta','W_B_phi','W_B_E','W_B_m']
header += ['b_B_pt','b_B_eta','b_B_phi','b_B_E','b_B_m']
header += ['Wjj_B_dR','tWb_B_dR'] #dR = sqrt(phi^2 + eta^2)
header += ['Wjj_B_deta','Wjj_B_dphi','tWb_B_deta','tWb_B_dphi']
# lab frame
header += ['W_pt','W_eta','W_phi','W_E','W_m']
header += ['b_pt','b_eta','b_phi','b_E','b_m']
header += ['Wjj_dR','tWb_dR']
header += ['Wjj_deta','Wjj_dphi','tWb_deta','tWb_dphi']
header += ['t_pt','t_eta','t_phi','t_E','t_m']

header += ['btag1', 'btag2', 'btag3'] #binary representation of likelihood of the jet being an actual bjet

df = pd.read_csv('~/projects/top-reco-tests/samples/result.csv', names=header, delimiter=' ', skiprows=1)

In [3]:
pos_class = df[df['label'] == 1]
neg_class = df[df['label'] == 0]
print(neg_class.shape)
neg_class = neg_class.sample(frac=0.1)
neg_class.shape
final_df = pd.concat([neg_class, pos_class])

(13759043, 43)


In [4]:
y = final_df['label']
X = final_df.drop('label', axis=1).drop('rndm', axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.15, shuffle=True)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.15, shuffle=True)

In [5]:
pca = PCA(n_components=5)
X_new = pca.fit_transform(X_train)


In [6]:
new_df = pd.DataFrame(X_new)
new_df.head()

,0,1,2,3,4
0,-489.675385,-40.086662,-31.844030,23.772749,9.474343
1,-343.669629,-147.492160,-35.906317,-17.417064,-20.550159
2,-607.255444,-79.614807,-31.900690,-70.721195,45.962700
3,-292.530589,146.645547,-121.094885,-0.032460,-1.289741
4,627.297041,346.720390,591.942761,259.849616,-299.140063


In [7]:
classifier = SVC()
classifier.fit(new_df[0:30], y_train[0:30])

/global/homes/r/rahuld/miniconda2/envs/py3.6/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='rbf', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [8]:
X_test_new = pd.DataFrame(pca.transform(X_test))
print(classifier.score(X_test_new[0:30], y_test[0:30]))

0.4666666666666667


In [9]:
predictions = []
for index in range(300):
    predictions.append(classifier.predict(X_train.iloc[int(np.argwhere(y_train == 1)[index])]))

/global/homes/r/rahuld/miniconda2/envs/py3.6/lib/python3.6/site-packages/numpy/core/fromnumeric.py:51: FutureWarning: Series.nonzero() is deprecated and will be removed in a future version.Use Series.to_numpy().nonzero() instead
  return getattr(obj, method)(*args, **kwds)


ValueError: Expected 2D array, got 1D array instead:
array=[ 1.90507328e-05  6.36006357e+01 -2.53303206e-01 -2.22701935e+00
  1.00401420e+02  7.59622649e+01  6.36006357e+01  2.53303206e-01
  9.14573307e-01  6.62552319e+01  8.92055371e+00  1.79259159e+00
  3.18217763e+00 -5.46613478e-01  1.70721947e+00 -5.06606411e-01
  3.14159265e+00  1.10153059e+02 -8.17065348e-02 -3.00898603e+00
  1.34108712e+02  7.59622649e+01  7.21599141e+01  2.82830566e-01
  1.62302661e+00  7.55935418e+01  8.92055371e+00  1.21861106e+00
  1.69093420e+00 -4.71812531e-01  1.12356836e+00 -3.64537101e-01
  1.65117267e+00  1.26745258e+02  9.19603602e-02  2.67074233e+00
  2.09702253e+02  1.66656652e+02  5.00000000e+00  1.00000000e+00
  1.00000000e+00].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [ ]:
sum(predictions)

In [12]:
pickle.dump(pca, open('models/pca_model.sav', 'wb'))

In [ ]:
pickle.